In [52]:
import medmnist
from medmnist import OrganAMNIST, INFO, Evaluator
import random
from timeit import default_timer as timer
import numpy as np
import scipy
import sklearn.neighbors
import sklearn.metrics
import skimage.transform
import skimage.util
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix
import torch
import torch.nn as nn
from torch import optim, inference_mode
from torch.utils.data import Subset, DataLoader
import torchvision
from torchvision import transforms
from torchmetrics import ConfusionMatrix
from tqdm.auto import tqdm


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
data_flag = 'organamnist'
info = INFO[data_flag]
DataClass = getattr(medmnist, info['python_class'])

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomRotation(degrees=15),  # Rotate image by up to 15 degrees
    transforms.
    transforms.Normalize(mean=[.5], std=[.5])
])

eval_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

train_data = DataClass(split='train', transform=train_transform, download=True)
val_data = DataClass(split='val', transform=eval_transform, download=True)
test_data = DataClass(split='test', transform=eval_transform, download=True)

# change data into dataloader form
BATCH_SIZE = 128
train_dataloader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(dataset=val_data, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)

# understanding the dataset
# Number of image channels
n_channels = info['n_channels']
# print(f"number of channels: {n_channels}")

# Number of classes
n_classes = len(info['label'])
# print(f"number of classes: {n_classes}")

# Get the class names from the dataset
class_names = info['label']
# print(f"class names: {class_names}")


# define model
class cnn(torch.nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units, 
                      kernel_size=3),
            nn.BatchNorm2d(hidden_units),
            nn.ReLU())

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units, 
                      kernel_size=3),
            nn.BatchNorm2d(hidden_units),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, 
                         stride=2))

        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units*4, 
                      kernel_size=3),
            nn.BatchNorm2d(hidden_units*4),
            nn.ReLU())
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units*4, 
                      out_channels=hidden_units*4, 
                      kernel_size=3),
            nn.BatchNorm2d(hidden_units*4),
            nn.ReLU())

        self.layer5 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units*4, 
                      out_channels=hidden_units*4, 
                      kernel_size=3,
                      padding=1),
            nn.BatchNorm2d(hidden_units*4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, 
                         stride=2))

        self.fc = nn.Sequential(
            nn.Linear(hidden_units*4 * 4 * 4, hidden_units*8),
            nn.ReLU(),
            nn.Linear(hidden_units*8, hidden_units*8),
            nn.ReLU(),
            nn.Linear(hidden_units*8, n_classes))

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [63]:

# define training loop helper functions
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    
    train_loss, train_acc = 0, 0
    model.to(device)
    
    for batch, (X, y) in enumerate(data_loader):
        # need to change target shape for this medmnist data 
        # y = y.squeeze().long()
        y = y.view(-1).long()

        
        # Send data to selected device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. loss and accuracy
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1))

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch 
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    
    return train_loss, train_acc

def test_step(data_loader: torch.utils.data.DataLoader,
            model: torch.nn.Module,
            loss_fn: torch.nn.Module,
            accuracy_fn,
            device: torch.device = device):

    test_loss, test_acc = 0, 0
    model.to(device)

    model.eval() # eval mode for testing
    with torch.inference_mode(): # Inference context manager
        for X, y in data_loader:
            # need to change target shape for this medmnist data 
            # y = y.squeeze().long()
            y = y.view(-1).long()

            
            # Send data to selected device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred = model(X)

            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                                    y_pred=test_pred.argmax(dim=1))

        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        
        return test_loss, test_acc
    
def eval_func(data_loader: torch.utils.data.DataLoader,
            model: torch.nn.Module,
            loss_fn: torch.nn.Module,
            accuracy_fn,
            device: torch.device = device):

    eval_loss, eval_acc = 0, 0
    model.to(device)

    model.eval()
    y_preds = []
    y_targets = []
    with torch.inference_mode():
        for batch, (X, y) in tqdm(enumerate(data_loader)):
            # need to change target shape for this medmnist data 
            y = y.squeeze().long()
            
            # Send data to selected device
            X, y = X.to(device), y.to(device)
            
            # Forward pass
            eval_pred = model(X)
            
            # Find loss and accuracy
            eval_loss += loss_fn(eval_pred, y)
            eval_acc += accuracy_fn(y_true=y,
                                    y_pred=eval_pred.argmax(dim=1))

            # Add prediction and target labels to list
            eval_labels = torch.argmax(torch.softmax(eval_pred, dim=1), dim=1)
            y_preds.append(eval_labels)
            y_targets.append(y)

        # Scale loss and acc 
        eval_loss /= len(data_loader)
        eval_acc /= len(data_loader)
        
        # Put predictions on CPU for evaluation
        y_preds=torch.cat(y_preds).cpu() 
        y_targets=torch.cat(y_targets).cpu() 
        
        return {"model_name": model.__class__.__name__, 
            "loss": eval_loss.item(),
            "accuracy": eval_acc,
            "predictions": y_preds,
            "targets": y_targets}
    
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() 
    acc = (correct / len(y_pred)) * 100 
    return acc

def print_train_time(start: float, end: float, device: torch.device = None):
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time



In [65]:
#CNN with Adam optimizer and image preprocessing
device = "cuda" if torch.cuda.is_available() else "cpu"
data_flag = 'organamnist'
info = INFO[data_flag]
DataClass = getattr(medmnist, info['python_class'])

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomRotation(degrees=15),  # Rotate image by up to 15 degrees
    transforms.Normalize(mean=[.5], std=[.5])
])

eval_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

train_data = DataClass(split='train', transform=train_transform, download=True)
val_data = DataClass(split='val', transform=eval_transform, download=True)
test_data = DataClass(split='test', transform=eval_transform, download=True)

# change data into dataloader form
BATCH_SIZE = 128
train_dataloader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(dataset=val_data, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)


# define Model
model = cnn(input_shape=n_channels, 
                     hidden_units=16,
                     output_shape=n_classes).to(device)


# setup loss and optimizer
loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# view Model
# model

# test performance
torch.manual_seed(42)

# measure Time
train_time_start_model = timer()
iteration_loss_list = []
iteration_accuracy_list = []

# set parameters
epochs = 10
best_loss = 10

# call train and test function
for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(data_loader=train_dataloader,
                                       model=model,
                                       loss_fn = loss_fn,
                                       optimizer=optimizer,
                                       accuracy_fn=accuracy_fn,
                                       device=device)
    
    test_loss, test_acc = test_step(data_loader=test_dataloader,
                                    model=model,
                                    loss_fn=loss_fn,
                                    accuracy_fn=accuracy_fn,
                                    device=device)
    
    for iteration, (x, y) in enumerate(train_dataloader):
        iteration_loss_list.append(train_loss.item())
        iteration_accuracy_list.append(train_acc)

    
    print(f"Epoch: {epoch} | Training loss: {train_loss:.3f} | Training acc: {train_acc:.2f} | Test loss: {test_loss:.3f} | Test acc: {test_acc:.2f}")

    # save best model instance

    if test_loss < best_loss:
        best_loss = test_loss
        print(f"Saving best model for epoch: {epoch}")
        torch.save(obj=model.state_dict(), 
                   f="./model.pth")     
train_time_end_model = timer()
total_train_time_model = print_train_time(start=train_time_start_model,
                                           end=train_time_end_model,
                                           device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0 | Training loss: 0.582 | Training acc: 79.12 | Test loss: 0.567 | Test acc: 81.98
Saving best model for epoch: 0
Epoch: 1 | Training loss: 0.329 | Training acc: 88.47 | Test loss: 0.463 | Test acc: 85.37
Saving best model for epoch: 1
Epoch: 2 | Training loss: 0.172 | Training acc: 94.02 | Test loss: 0.518 | Test acc: 84.51
Epoch: 3 | Training loss: 0.129 | Training acc: 95.45 | Test loss: 0.933 | Test acc: 76.87
Epoch: 4 | Training loss: 0.174 | Training acc: 94.20 | Test loss: 0.806 | Test acc: 80.59
Epoch: 5 | Training loss: 0.117 | Training acc: 96.06 | Test loss: 0.493 | Test acc: 87.34
Epoch: 6 | Training loss: 0.071 | Training acc: 97.59 | Test loss: 0.520 | Test acc: 86.70
Epoch: 7 | Training loss: 0.057 | Training acc: 98.00 | Test loss: 0.750 | Test acc: 83.69
Epoch: 8 | Training loss: 0.061 | Training acc: 97.93 | Test loss: 0.595 | Test acc: 86.44
Epoch: 9 | Training loss: 0.062 | Training acc: 97.84 | Test loss: 0.527 | Test acc: 87.36
Train time on cpu: 270.369 s

In [66]:
#CNN with SGD optimizer and image preprocessing
device = "cuda" if torch.cuda.is_available() else "cpu"
data_flag = 'organamnist'
info = INFO[data_flag]
DataClass = getattr(medmnist, info['python_class'])

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomRotation(degrees=15),  # Rotate image by up to 15 degrees
    transforms.Normalize(mean=[.5], std=[.5])
])

eval_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

train_data = DataClass(split='train', transform=train_transform, download=True)
val_data = DataClass(split='val', transform=eval_transform, download=True)
test_data = DataClass(split='test', transform=eval_transform, download=True)

# change data into dataloader form
BATCH_SIZE = 128
train_dataloader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(dataset=val_data, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)


# define Model
model = cnn(input_shape=n_channels, 
                     hidden_units=16,
                     output_shape=n_classes).to(device)


# setup loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# view Model
# model

# test performance
torch.manual_seed(42)

# measure Time
train_time_start_model = timer()
iteration_loss_list = []
iteration_accuracy_list = []

# set parameters
epochs = 10
best_loss = 10

# call train and test function
for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(data_loader=train_dataloader,
                                       model=model,
                                       loss_fn = loss_fn,
                                       optimizer=optimizer,
                                       accuracy_fn=accuracy_fn,
                                       device=device)
    
    test_loss, test_acc = test_step(data_loader=test_dataloader,
                                    model=model,
                                    loss_fn=loss_fn,
                                    accuracy_fn=accuracy_fn,
                                    device=device)
    
    for iteration, (x, y) in enumerate(train_dataloader):
        iteration_loss_list.append(train_loss.item())
        iteration_accuracy_list.append(train_acc)

    
    print(f"Epoch: {epoch} | Training loss: {train_loss:.3f} | Training acc: {train_acc:.2f} | Test loss: {test_loss:.3f} | Test acc: {test_acc:.2f}")

    # save best model instance

    if test_loss < best_loss:
        best_loss = test_loss
        print(f"Saving best model for epoch: {epoch}")
        torch.save(obj=model.state_dict(), 
                   f="./model.pth")     
train_time_end_model = timer()
total_train_time_model = print_train_time(start=train_time_start_model,
                                           end=train_time_end_model,
                                           device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0 | Training loss: 1.652 | Training acc: 41.80 | Test loss: 1.271 | Test acc: 59.63
Saving best model for epoch: 0
Epoch: 1 | Training loss: 0.720 | Training acc: 75.12 | Test loss: 0.810 | Test acc: 72.57
Saving best model for epoch: 1
Epoch: 2 | Training loss: 0.426 | Training acc: 85.38 | Test loss: 0.702 | Test acc: 76.90
Saving best model for epoch: 2
Epoch: 3 | Training loss: 0.311 | Training acc: 88.83 | Test loss: 1.306 | Test acc: 62.15
Epoch: 4 | Training loss: 0.305 | Training acc: 89.44 | Test loss: 0.706 | Test acc: 79.22
Epoch: 5 | Training loss: 0.204 | Training acc: 92.96 | Test loss: 0.668 | Test acc: 80.58
Saving best model for epoch: 5
Epoch: 6 | Training loss: 0.161 | Training acc: 94.51 | Test loss: 0.581 | Test acc: 83.22
Saving best model for epoch: 6
Epoch: 7 | Training loss: 0.136 | Training acc: 95.29 | Test loss: 0.614 | Test acc: 82.64
Epoch: 8 | Training loss: 0.117 | Training acc: 95.86 | Test loss: 0.580 | Test acc: 82.89
Saving best model for epoc

In [67]:
#CNN with SGD optimizer
device = "cuda" if torch.cuda.is_available() else "cpu"
data_flag = 'organamnist'
info = INFO[data_flag]
DataClass = getattr(medmnist, info['python_class'])

data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

train_data = DataClass(split='train', transform=data_transform, download=True)
val_data = DataClass(split='val', transform=data_transform, download=True)
test_data = DataClass(split='test', transform=data_transform, download=True)

# change data into dataloader form
BATCH_SIZE = 128
train_dataloader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(dataset=val_data, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)


# define Model
model = cnn(input_shape=n_channels, 
                     hidden_units=16,
                     output_shape=n_classes).to(device)


# setup loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# view Model
# model

# test performance
torch.manual_seed(42)

# measure Time
train_time_start_model = timer()
iteration_loss_list = []
iteration_accuracy_list = []

# set parameters
epochs = 10
best_loss = 10

# call train and test function
for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(data_loader=train_dataloader,
                                       model=model,
                                       loss_fn = loss_fn,
                                       optimizer=optimizer,
                                       accuracy_fn=accuracy_fn,
                                       device=device)
    
    test_loss, test_acc = test_step(data_loader=test_dataloader,
                                    model=model,
                                    loss_fn=loss_fn,
                                    accuracy_fn=accuracy_fn,
                                    device=device)
    
    for iteration, (x, y) in enumerate(train_dataloader):
        iteration_loss_list.append(train_loss.item())
        iteration_accuracy_list.append(train_acc)

    
    print(f"Epoch: {epoch} | Training loss: {train_loss:.3f} | Training acc: {train_acc:.2f} | Test loss: {test_loss:.3f} | Test acc: {test_acc:.2f}")

    # save best model instance

    if test_loss < best_loss:
        best_loss = test_loss
        print(f"Saving best model for epoch: {epoch}")
        torch.save(obj=model.state_dict(), 
                   f="./model.pth")     
train_time_end_model = timer()
total_train_time_model = print_train_time(start=train_time_start_model,
                                           end=train_time_end_model,
                                           device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0 | Training loss: 1.646 | Training acc: 41.98 | Test loss: 1.264 | Test acc: 59.67
Saving best model for epoch: 0
Epoch: 1 | Training loss: 0.684 | Training acc: 76.36 | Test loss: 0.759 | Test acc: 74.99
Saving best model for epoch: 1
Epoch: 2 | Training loss: 0.374 | Training acc: 87.38 | Test loss: 4.176 | Test acc: 35.00
Epoch: 3 | Training loss: 0.503 | Training acc: 83.82 | Test loss: 0.643 | Test acc: 79.71
Saving best model for epoch: 3
Epoch: 4 | Training loss: 0.226 | Training acc: 92.34 | Test loss: 0.698 | Test acc: 79.31
Epoch: 5 | Training loss: 0.160 | Training acc: 94.57 | Test loss: 0.625 | Test acc: 82.32
Saving best model for epoch: 5
Epoch: 6 | Training loss: 0.115 | Training acc: 96.03 | Test loss: 0.601 | Test acc: 83.15
Saving best model for epoch: 6
Epoch: 7 | Training loss: 0.090 | Training acc: 97.02 | Test loss: 0.660 | Test acc: 83.56
Epoch: 8 | Training loss: 0.062 | Training acc: 97.94 | Test loss: 0.677 | Test acc: 83.98
Epoch: 9 | Training loss: 

In [68]:
#CNN with Adam optimizer
device = "cuda" if torch.cuda.is_available() else "cpu"
data_flag = 'organamnist'
info = INFO[data_flag]
DataClass = getattr(medmnist, info['python_class'])

data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

train_data = DataClass(split='train', transform=data_transform, download=True)
val_data = DataClass(split='val', transform=data_transform, download=True)
test_data = DataClass(split='test', transform=data_transform, download=True)

# change data into dataloader form
BATCH_SIZE = 128
train_dataloader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(dataset=val_data, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)


# define Model
model = cnn(input_shape=n_channels, 
                     hidden_units=16,
                     output_shape=n_classes).to(device)


# setup loss and optimizer
loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# view Model
# model

# test performance
torch.manual_seed(42)

# measure Time
train_time_start_model = timer()
iteration_loss_list = []
iteration_accuracy_list = []

# set parameters
epochs = 10
best_loss = 10

# call train and test function
for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(data_loader=train_dataloader,
                                       model=model,
                                       loss_fn = loss_fn,
                                       optimizer=optimizer,
                                       accuracy_fn=accuracy_fn,
                                       device=device)
    
    test_loss, test_acc = test_step(data_loader=test_dataloader,
                                    model=model,
                                    loss_fn=loss_fn,
                                    accuracy_fn=accuracy_fn,
                                    device=device)
    
    for iteration, (x, y) in enumerate(train_dataloader):
        iteration_loss_list.append(train_loss.item())
        iteration_accuracy_list.append(train_acc)

    
    print(f"Epoch: {epoch} | Training loss: {train_loss:.3f} | Training acc: {train_acc:.2f} | Test loss: {test_loss:.3f} | Test acc: {test_acc:.2f}")

    # save best model instance

    if test_loss < best_loss:
        best_loss = test_loss
        print(f"Saving best model for epoch: {epoch}")
        torch.save(obj=model.state_dict(), 
                   f="./model.pth")     
train_time_end_model = timer()
total_train_time_model = print_train_time(start=train_time_start_model,
                                           end=train_time_end_model,
                                           device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0 | Training loss: 0.535 | Training acc: 81.12 | Test loss: 0.753 | Test acc: 78.71
Saving best model for epoch: 0
Epoch: 1 | Training loss: 0.233 | Training acc: 92.20 | Test loss: 0.614 | Test acc: 82.08
Saving best model for epoch: 1
Epoch: 2 | Training loss: 0.102 | Training acc: 96.52 | Test loss: 0.761 | Test acc: 80.36
Epoch: 3 | Training loss: 0.080 | Training acc: 97.27 | Test loss: 0.685 | Test acc: 84.30
Epoch: 4 | Training loss: 0.047 | Training acc: 98.41 | Test loss: 0.793 | Test acc: 84.81
Epoch: 5 | Training loss: 0.044 | Training acc: 98.53 | Test loss: 0.667 | Test acc: 85.81
Epoch: 6 | Training loss: 0.031 | Training acc: 98.94 | Test loss: 0.687 | Test acc: 86.01
Epoch: 7 | Training loss: 0.026 | Training acc: 99.11 | Test loss: 0.699 | Test acc: 85.44
Epoch: 8 | Training loss: 0.030 | Training acc: 99.03 | Test loss: 0.792 | Test acc: 85.76
Epoch: 9 | Training loss: 0.021 | Training acc: 99.31 | Test loss: 0.750 | Test acc: 86.50
Train time on cpu: 244.666 s